# Exercise 5 - Single Spike Train Analysis
Guy Singer
11.12.24

### Table of Contents

* [Part I: Spike Train Characteristics](#part1-spike-train)
   * [Last Week's Findings](#last-week)
   * [Interspike Interval Distribution](#interspike-interval)
   * [Poisson Spike Generator](#poisson-generator)
   * [Hazard and Survival Functions](#hazard-survival)

* [Part II: Cross- and Auto-correlation](#part2-correlation)
   * [Cross-correlation](#cross-correlation)
   * [Auto-correlation](#auto-correlation)
   * [Meaning of τ = 0 Value](#tau-zero)
   * [Predicting Neuron Type](#predict-neuron)
   * [Y-axis Values in Correlations](#y-axis)
   * [Non-stationarity](#non-stationarity)

## Part I: Spike Train Characteristics <a id='part1-spike-train'></a>

### Last Week's Findings <a id='last-week'></a>

Last week we deduced the expression for $P_T(n)$ or the model we use for the probability that any sequence of $n$ spikes occurs within a trial of duration $T$. After some inspection we came up with the following result:

$P_T(n) = \lim_{\Delta t \rightarrow 0} \binom{M}{n} (r \Delta t)^n (1 - r\Delta t)^{M - n}$

which can be simplified to

$P_T(n) = \frac{(rT)^n}{n!}e^{-rT}$

which also describes a Poisson process with a parameter of $rT$. From $P_T(n)$ we can deduce the chance for a *specific* spike train, i.e. a train with spikes at given times $\{t_1, \ldots, t_n\}$:

$P(t_1,\ldots,t_n) = n! P_T(n) \left( \frac{\Delta t}{T} \right)^n$

### Interspike Interval Distribution <a id='interspike-interval'></a>

A useful metric for spike trains in their interspike interval, i.e. the time between adjacent spikes. This statistical property is a way for us to discriminate between different neuronal behaviors in a quantitative manner. To arrive at that distribution, we assume that a spike occurred at time $t_i$ and we ask ourselves what is the probability (in an HPP) that another spike will occur in the short interval $t_i + \tau \leq t_{i+1} < t_i + \tau + \Delta t$ (figure 1, assuming $\Delta t$ is very small). That is, what is the chance for a spike $\tau$ seconds after the previous, with an interval of $\Delta t$ after it.

![figure](isi.png)
*Figure 1: Spike timing definitions.*

To answer, we need to see what is the probability that no spikes will fire for $\tau$ seconds, and then follow up with the probability that a spike will occur in the subsequent $\Delta t$ interval. The chance for $n=0$ spikes in the first interval is:

$P_\tau(n=0) = \frac{(r\tau)^0}{0!}e^{-r\tau} = e^{-r\tau}$

and the chance for a spike in a short interval $\Delta t$ is by definition $r \Delta t$. Thus, the probability of an interspike interval with a value of $\tau$ is:

$P(t_i + \tau \leq t_{i+1} < t_i + \tau + \Delta t) = r\Delta t e^{-rT}$

The PDF of interspike intervals is the same expression as above without the $\Delta t$, making it an exponential probability density function. From this inter-spike interval one can create the time interval histogram (TIH), which has an exponential trend, as seen in figure 2. These quantities, the ISI and the TIH indicate that long ISIs are generally less likely, at least in the case of the simple Poisson neuron. Any divergence from a simple exponential may indicate of some underlying property that will probably be of interest. For example, an oscillatory neuron will have a peak in its TIH at that expected time interval.

![figure](tih.png)
*Figure 2: Time interval histogram. X-axis is time in ms, Y-axis is percentage of total spikes. From Izhar Bar-Gad lectures.*

One notable difference we already see between a pure exponential and the TIH depicted above, is the number of ISIs in the lower-valued bins. Instead of the TIH obeying a simple exponential decay, it starts off very low, and only after about 10 ms it starts the expected decay process. This is due to the refractory period of real neurons, that was not accounted for during our derivation process. A more accurate function that describes the measured distribution is the Gamma function.

### Question
What is the expected mean interspike interval?

### Answer
We'll compute the expected value of the PDF to find the mean interval:

$\begin{aligned}
\left<\tau\right> & = \int_0^\infty \tau p(\tau) d\tau \\
& = \int_0^\infty \tau r e^{-r\tau} d\tau \\
& = r \int_0^\infty \tau e^{-r\tau} d \tau \text{\ \ \ (Gamma function)}\\
& = r \left[ \frac{1}{r^2} \right] = \frac{1}{r}
\end{aligned}$

This result is not surprising - for an average firing rate of $r$, the mean interval between spikes is $\frac{1}{r}$.